In [1]:
import numpy
from src.models.conv_model import build_model as build_conv_model
import tensorflow as tf
from src.evaluator.evaluator import Evaluator
from src.models.conv_model import correlation_coefficient_loss, pearson_r
import math
from src.data_loader.RNASeqLoader import RNASeqDataGenerator 
dependencies = {
    'correlation_coefficient_loss': correlation_coefficient_loss,
    'pearson_r': pearson_r
}
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

Using TensorFlow backend.


In [2]:
train_generator = RNASeqDataGenerator("data/processed/cDNA-ABE/train_data.hdf5", 512, n_around_center=25)
validation_generator = RNASeqDataGenerator("data/processed/cDNA-ABE/validation_data.hdf5", 512, n_around_center=25)

#model = tf.keras.models.load_model('models/cDNA-ABE/logs_nostruct/model_ckpt/1611875098.0045986.h5', custom_objects=dependencies)

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv1D, Dropout, Flatten, BatchNormalization, MaxPool1D, Activation
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf

def custom_model(dropout):
    model = Sequential()
    model.add(Input(shape=(51,4)))
    model.add(Conv1D(filters=128, kernel_size=32, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Conv1D(filters=128, kernel_size=32, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Conv1D(filters=128, kernel_size=32, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Conv1D(filters=128, kernel_size=32, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Conv1D(filters=64, kernel_size=32, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Conv1D(filters=64, kernel_size=32, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Conv1D(filters=32, kernel_size=16, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Conv1D(filters=32, kernel_size=16, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Conv1D(filters=32, kernel_size=16, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Flatten())
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = tf.keras.optimizers.RMSprop(decay=0.0005)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['binary_crossentropy', 'mse', correlation_coefficient_loss, pearson_r])
    return model 

In [4]:
model = custom_model(0.0)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
print(model.summary())
checkpoint_filepath = "models/cDNA-ABE/2.10.21/model-{epoch:02d}-{val_loss:.4f}.h5"

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    #monitor='val_pearson_r',
    #mode='max',
    save_best_only=False)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3,
                              patience=4, min_lr=0.00001)

history = model.fit(x=train_generator, epochs=10, validation_data=validation_generator, callbacks=[model_checkpoint_callback], use_multiprocessing=True, workers=5)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 51, 128)           16512     
_________________________________________________________________
batch_normalization (BatchNo (None, 51, 128)           512       
_________________________________________________________________
activation (Activation)      (None, 51, 128)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 51, 128)           524416    
_________________________________________________________________
batch_normalization_1 (Batch (None, 51, 128)           512       
_________________________________________________________________
activation_1 (Activation)    (None, 51, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 51, 128)           5